In [ ]:
import pept
import numpy as np
from pept.plots import PlotlyGrapher, PlotlyGrapher2D
from pept.tracking import *
import matplotlib.pyplot as plt
import matplotlib as mpl
import plotly
import plotly.express as px
import plotly.io as pio
from mpl_toolkits.mplot3d import Axes3D

In [ ]:
# Initialise data from a file and set sample size and overlap
filepath = ".da01" # use da* to load multiple
max_tracers = 1

lors = pept.scanners.adac_forte(
    filepath,
    sample_size = 200 * max_tracers,
    overlap = 150 * max_tracers,
)

# Select only the first 1000 samples of LoRs for testing
#lors = lors[:1000]

# Create PEPT-ML processing pipeline
pipeline = pept.Pipeline([
    OptimizeWindow(200, overlap = 0.5) + Debug(1),

    # First pass of clustering
    Cutpoints(max_distance = 0.2),
    HDBSCAN(true_fraction = 0.15, max_tracers = max_tracers),
    SplitLabels() + Centroids(cluster_size = True, error = True),

    # Remove erroneous points
    Condition("cluster_size > 30, error < 20"),

    # Second pass of clustering
    Stack(sample_size = 30 * max_tracers, overlap = 30 * max_tracers - 1),
    HDBSCAN(true_fraction = 0.6, max_tracers = max_tracers),
    SplitLabels() + Centroids(),

    # Remove sparse points in time
    OutOfViewFilter(200.),

    # Trajectory separation
    Segregate(window = 20 * max_tracers, cut_distance = 10),

    # Velocity computation
    Velocity(11),
    Velocity(11, absolute = True),

    # Cutoff points outside this region
    Condition("x > 100, x < 400"),
    Condition("y > 150, y < 300"),

    Center(), # Center model

    Stack(),
])

# Process all samples in `lors` in parallel, using `max_workers` threads
trajectories = pipeline.fit(lors)

# Save as a fast binary; you can load them back with `pept.load("path")`
trajectories.save(filepath + "fullruns.pickle")

# Plot points
PlotlyGrapher().add_points(trajectories).show()

In [ ]:
# load in pickle files
data = pept.load(".pickle")

# Compute Streamlines
split_pipe = pept.Pipeline([

    Segregate(window = 10, cut_distance = 0.4),  # Appends label column
    GroupBy("label"),                           # Splits into samples
    Stack(),
])

streamlines = split_pipe.fit(data) 

In [ ]:
# plot stream lines
fig, ax = plt.subplots()
for i in range(len(streamlines)):
    if len(streamlines[i]['x']) > 50: # remove streamlines less than 50 points in length
        x_stream = streamlines[i]['x']
        y_stream = streamlines[i]['y']
        ax.plot(x_stream,y_stream, linewidth=0.2)
#plt.xlim(130,360)
plt.ylim(-20,60)
plt.xlabel("x (mm)")
plt.ylabel("y (mm)")
plt.figure(dpi=500)
fig.savefig(".png", dpi=500)